In [153]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

# Set up Selenium webdriver
service = Service('chromedriver_win32\chromedriver.exe')  # Replace with the path to your chromedriver executable
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 10)

# Open the URL
driver.get('https://sis.punjab.gov.pk/dashboard')
time.sleep(5)

# Click on "Sanctioned Posts"
sanctioned_posts = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[text()="Sanctioned Posts"]')))
sanctioned_posts.click()
time.sleep(5)

# Wait for the page to load
wait.until(EC.presence_of_element_located((By.XPATH, '//a[text()="Non-Teaching"]')))

# Click on "Non-Teaching"
non_teaching = driver.find_element(By.XPATH, '//a[text()="Non-Teaching"]')
non_teaching.click()

time.sleep(5)

# Get the page source after all the interactions
page_source = driver.page_source

# Close the browser
# driver.quit()

# Process the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table by class name
table = soup.find('table', class_='table table-striped table-bordered table-hover table-highlight table-checkable sanctioned-post-table mt-10')

table_header = [cell.get_text(strip=True) for cell in table.find_all('th')]

# Extract table data into a list of lists
table_data = []
for row in table.find_all('tr'):
    row_data = [cell.get_text(strip=True) for cell in row.find_all('td')]
    table_data.append(row_data)

table_data[0] = table_header

# Create a DataFrame from the table data
df = pd.DataFrame(table_data[1:], columns=table_data[0])

col_name = list(df.iloc[:, 1])
col_name = col_name[:-1]

VP_col = []
for col in col_name:
    VP_col.extend([col + '_total', col + '_vacant', col + '_filled'])

vacancy = pd.read_csv('C:/Users/lasan/Downloads/SIS/Vacancy.csv',low_memory=False, index_col=0)


# Check column names and match with VP_col list
for col in vacancy.columns:
    if col not in VP_col:
        VP_col.append(col)



def add_column_if_not_exists(df, column_name, default_value=None):
    if column_name not in df.columns:
        df[column_name] = default_value


for i in VP_col:
    add_column_if_not_exists(vacancy, i, default_value=0)

vacancy.to_csv('C:/Users/lasan/Downloads/SIS/Vacancy.csv',mode='w', index=True, header=True)

In [264]:

emis_code = pd.read_csv('C:/Users/lasan/Downloads/SIS/Vacancy.csv')
emis_code = list(emis_code.iloc[:,0])
for i, code in enumerate(emis_code):
    # select emis code box of senctioned posts tab where non teaching
    text_box = driver.find_element(By.ID,'sanctioned_posts_form')
    text_box = text_box.find_element(By.ID,'emis_code')
    text_box.click()
    text_box.clear()
    # send EMIS code to box
    text_box.send_keys(code)
    # press enter
    text_box.send_keys(Keys.RETURN)
    time.sleep(2)
    page_source = driver.page_source

    soup1 = BeautifulSoup(page_source, 'html.parser')

    # Find the table by class name
    table2 = soup1.find('div', id="non_teaching_sanctioned_post_tab_data")

    # table headers of selected emis code vacancy position
    table_header1 = [cell.get_text(strip=True) for cell in table2.find_all('th')]

    # Extract table data into a list of lists
    table_data1 = []
    for row in table2.find_all('tr'):
        row_data = [cell.get_text(strip=True) for cell in row.find_all('td')]
        table_data1.append(row_data)
    # set table header
    table_data1[0] = table_header1

    # Create a DataFrame from the table data and skip the # column
    df1 = pd.DataFrame(table_data1[1:-1], columns=table_data1[0])

    # column name of form posts column
    col_name1 = list(df1.iloc[:, 1])

    # convert the post columns values to columns names of new dataset
    VP_col1 = []
    for col in col_name1:
        VP_col1.extend([col + '_vacant', col + '_filled',col + '_total'] )
    # convert all values to a single list 1 x 1 shape
    VP_Val= df1.iloc[:,2:].values.reshape(1,-1)
    # read updated data from csv file
    df_previous = pd.read_csv('Vacancy.csv')
    # create dataframe of selected emis
    VP_df = pd.DataFrame(VP_Val,columns=VP_col1)
    # filter the row of selected columns from existing
    filtered_df  = df_previous[df_previous.loc[:,'EMIS']==code]

    #SET all column values to zero of selected EMIS
    df_previous.loc[filtered_df.index,filtered_df.columns[3:] ] = 0
    # Update new values
    df_previous.loc[filtered_df.index, VP_col1] = VP_Val


# Save the updated DataFrame back to the CSV file
    df_previous.to_csv('Vacancy.csv', index=False)


